In [1]:
# Import ODE Solver
include("../ode_solver.jl")
using PlotlyJS

WebIO._IJuliaInit()

Function Definition

In [2]:
"""
dx/dt = x
"""

function f(x, t)
    return x
end

f (generic function with 1 method)

Plot of approximation methods compared to solution

In [5]:
t = [0, 3]
x0 = [1]
deltat_max = 0.01

@time s1 = solve_ode(f, x0, t, "euler", deltat_max)[:,1]
@time s2 = solve_ode(f, x0, t, "heun3", deltat_max)[:,1]
@time s3 = solve_ode(f, x0, t, "ralston4", deltat_max)[:,1]
@time s4 = solve_ode(f, x0, t, "rk4", deltat_max)[:,1]
@time s5 = solve_ode(f, x0, t, "three_eighths_rule", deltat_max)[:,1]

# Create traces
euler = scatter(x=t, y=s1, mode="lines", name="euler")
heun3 = scatter(x=t, y=s2, mode="lines", name="heun3")
ralston4 = scatter(x=t, y=s3, mode="lines", name="ralston4")
rk4 = scatter(x=t, y=s2, mode="lines", name="rk4")
three_eighths_rule = scatter(x=t, y=s5, mode="lines", name="three_eighths_rule")

solution = scatter(x=0:0.01:3, y=exp.(0:0.01:3), mode="lines", name="solution (e^(x))")

plot([euler, heun3, ralston4, rk4, three_eighths_rule, solution])

  0.045568 seconds (108.17 k allocations: 6.283 MiB, 23.66% gc time, 99.04% compilation time)
  0.000674 seconds (8.13 k allocations: 315.531 KiB)
  0.000743 seconds (12.35 k allocations: 550.688 KiB)
  0.000869 seconds (9.64 k allocations: 381.375 KiB)
  0.000598 seconds (10.84 k allocations: 456.641 KiB)


data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin and template"

Error Plot

In [24]:
x0 = [1]
t = [0 1]

# Euler estimate of x(1)
deltat_max = 0.001
solution = solve_ode(f, x0, t, "euler", deltat_max)
println("Euler approximation = ", solution[end][1])

# Heun3 estimate of x(1)
deltat_max = 0.001
solution = solve_ode(f, x0, t, "heun3", deltat_max)
println("Heun3 approximation = ", solution[end][1])

# Ralston4 estimate of x(1)
deltat_max = 0.001
solution = solve_ode(f, x0, t, "ralston4", deltat_max)
println("Ralston4 approximation = ", solution[end][1])

# RK4 estimate of x(1)
deltat_max = 0.001
solution = solve_ode(f, x0, t, "rk4", deltat_max)
println("RK4 approximation = ", solution[end][1])

# 3/8 Rule estimate of x(1)
deltat_max = 0.001
solution = solve_ode(f, x0, t, "three_eighths_rule", deltat_max)
println("3/8 Rule approximation = ", solution[end][1])

x0 = [1.]
t = [0, 1]
deltat_max = exp10.(range(-7,0,80))
real = ℯ

euler_error = Any[]
heun3_error = Any[]
ralston4_error = Any[]
rk4_error = Any[]
three_eighths_rule_error = Any[]

for value in deltat_max
    euler_sol = solve_ode(f, x0, t, "euler", value)[end][1];
    heun3_sol = solve_ode(f, x0, t, "heun3", value)[end][1];
    ralston4_sol = solve_ode(f, x0, t, "ralston4", value)[end][1];
    rk4_sol = solve_ode(f, x0, t, "rk4", value)[end][1];
    three_eighths_rule_sol = solve_ode(f, x0, t, "three_eighths_rule", value)[end][1];
    push!(euler_error, abs.(euler_sol .- real))
    push!(heun3_error, abs.(heun3_sol .- real))
    push!(ralston4_error, abs.(ralston4_sol .- real))
    push!(rk4_error, abs.(rk4_sol .- real))
    push!(three_eighths_rule_error, abs.(three_eighths_rule_sol .- real))   
end

Euler approximation = 2.7169239322358942
Heun3 approximation = 2.7182818283458725
Ralston4 approximation = 2.7182815566175837
RK4 approximation = 2.718281828459023
3/8 Rule approximation = 2.718281828459023


In [25]:
t1 = scatter(x=deltat_max, y=euler_error, mode="lines", name="Euler")
t2 = scatter(x=deltat_max, y=heun3_error, mode="lines", name="Heun3 ")
t3 = scatter(x=deltat_max, y=ralston4_error, mode="lines", name="Ralston 4")
t4 = scatter(x=deltat_max, y=rk4_error, mode="lines", name="RK 4")
t5 = scatter(x=deltat_max, y=three_eighths_rule_error, mode="lines", name="3/8 Rule")

layout = Layout(
    xaxis_type="log",
    xaxis_exponentformat="power",
    xaxis_title="Δt",
    yaxis_type="log",
    yaxis_exponentformat="power",
    yaxis_title="error",
    width=700, height=350,
    )
data = [t1, t4, t2, t3, t5]

plot(data, layout)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"

Timing

In [13]:
t = [0 1]
deltat_max = 0.01
method_error = 1
while method_error >= 2.0843238788259555e-6
    deltat_max = deltat_max .* 0.1 
    s1 = solve_ode(f, [1], t, "euler", deltat_max)[end][1]
    method_error = abs(ℯ .- s1)
end
deltat_max


rk4_deltat_max = 0.1
# method_error = 1
# while method_error >= 0.000001
#     rk4_deltat_max = rk4_deltat_max .* 0.1 
#     s1 = solve_ode(f, [1], t, rk4_step, rk4_deltat_max)[end][1]
#     method_error = abs(ℯ .- s1)
# end
[deltat_max, rk4_deltat_max]

2-element Vector{Float64}:
 1.0000000000000002e-6
 0.1

In [14]:
t = [0 1]

@time s1 = solve_ode(f, [1], t, euler_step, 1e-6 )[end][1];
println("Euler method error: ", abs(ℯ .- s1))
@time s2 = solve_ode(f, [1], t, rk4_step, 0.1)[end][1];
println("RK4 method error: ", abs(ℯ .- s2))


Method not assigned, please enter either:
        euler, heun3, ralston4, rk4, or three_eighths_rule
Defaulting to rk4_step.
  1.983270 seconds (32.13 M allocations: 1.214 GiB, 6.80% gc time, 2.57% compilation time)
Euler method error: 2.1465496047312627e-11
Method not assigned, please enter either:
        euler, heun3, ralston4, rk4, or three_eighths_rule
Defaulting to rk4_step.
  0.000527 seconds (397 allocations: 16.422 KiB)
RK4 method error: 2.0843238788259555e-6


ODE Systems Extensions

In [28]:
"""
System of equations definitions for
    d^2x/dt^2 = -x,
equivalent to the system of equations
    dx/dt = y and dy/dt = -x.
"""

function f2(u, t)

    if(!isapprox(length(u), 2.0; atol=eps(Float64), rtol=0))
        throw(error("Please make sure you have entered two initial conditions for the function."))
    end

    x = u[1]    
    y = u[2]
    
    x_dot = y
    y_dot = -x
    
    return [x_dot y_dot]
end

function f2_solution(u, t)

    c1 = u[2]
    c2 = u[1]

    x = c1*sin.(t) + c2*cos.(t)
    y = c1*cos.(t) - c2*sin.(t)

    return [x y]
end

f2_solution (generic function with 1 method)

Generate solution and estimate data for error plots

In [29]:
u = [0 0]
t = 0:0.1:10
real_t = 0:0.1:10
s = solve_ode(f2, u, t, "rk4", 0.001);
real_s = f2_solution(u, real_t);

x = s[:,1];
x_dot = s[:,2];
x_sol = real_s[:,1];
x_dot_sol = real_s[:,2];

Plot of solution and estimate for x versus t

In [37]:
u = [1 0]
t = 0:0.2:10
real_t = 0:0.1:10
s = solve_ode(f2, u, t, "rk4", 0.001);
real_s = f2_solution(u, real_t);

x = s[:,1];
x_dot = s[:,2];
x_sol = real_s[:,1];
x_dot_sol = real_s[:,2];

solution = scatter(x=real_t, y=x_sol, mode="lines",name="solution")
estimate = scatter(x=t, y=x, mode="lines",name="estimate")

layout = Layout(
    xaxis_title="time",
    yaxis_title="x",
    width=700, height=350,
    )

plot([solution, estimate], layout)

# include("../visualisation.jl")
# plot_ode(f2, u, t, ["x" "y"])

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"

Plot of solution and estimate for x versus x_dot

In [143]:
estimate = scatter(x=x_dot, y=x, mode="lines",name="estimate")
solution = scatter(x=x_dot_sol, y=x_sol, mode="lines",name="solution")
layout = Layout(
    yaxis=attr(scaleanchor = "x",scaleratio = 1),
    xaxis_title="x_dot",
    yaxis_title="x",
    width=700, height=350,
    )

plot([estimate, solution], layout)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"